In [1]:
import torch
import torch.nn as nn

In [2]:
import numpy as np

In [3]:
from itertools import permutations

In [4]:
model=torch.load("IN_best_dec10.pth",map_location=torch.device('cpu'))

In [5]:
params={}
processed=set()
w_cnt=0
b_cnt=0

for k in model.keys():
    ln=k.split(".")[2:]
    
    if "weight" in ln:
        n=f"w{w_cnt}"
        
        print(ln[:2],w_cnt)
        w_cnt+=1
        
    elif "bias" in ln:
        n=f"b{b_cnt}"
        b_cnt+=1
        
    else:
        continue
    

    lv=model[k]
    
    params[n]=lv

['edge_mlp', '0'] 0
['edge_mlp', '1'] 1
['edge_mlp', '3'] 2
['node_mlp_1', '0'] 3
['node_mlp_1', '1'] 4
['node_mlp_1', '3'] 5
['node_mlp_2', '0'] 6
['node_mlp_2', '1'] 7
['node_mlp_2', '3'] 8
['global_mlp', '0'] 9
['global_mlp', '1'] 10
['global_mlp', '3'] 11


In [6]:
for k in params.keys():
    print(k,params[k].shape)

w0 torch.Size([128, 96])
b0 torch.Size([128])
w1 torch.Size([128])
b1 torch.Size([128])
w2 torch.Size([128, 128])
b2 torch.Size([128])
w3 torch.Size([128, 176])
b3 torch.Size([128])
w4 torch.Size([128])
b4 torch.Size([128])
w5 torch.Size([128, 128])
b5 torch.Size([128])
w6 torch.Size([128, 176])
b6 torch.Size([128])
w7 torch.Size([128])
b7 torch.Size([128])
w8 torch.Size([128, 128])
b8 torch.Size([128])
w9 torch.Size([128, 128])
b9 torch.Size([128])
w10 torch.Size([128])
b10 torch.Size([128])
w11 torch.Size([2, 128])
b11 torch.Size([2])


In [81]:
# create dummy input 

n=3
x=torch.ones(n,48,dtype=torch.float64,requires_grad=True)
edge_index=torch.tensor(list(permutations(range(n),2))).T

In [82]:
row,col=edge_index

In [83]:
a=torch.cat([x[row],x[col]],1)
a.shape

torch.Size([6, 96])

In [84]:
# argument input

alpha=x[col]/(n-1)
beta=x[row]
a_p=torch.cat([alpha,beta,a],1)
a_p.shape # 48+48+48*2

torch.Size([6, 192])

edge block

In [85]:
w0=params["w0"]
w0.shape

torch.Size([128, 96])

In [86]:
w0_p=np.eye(48*2+w0.shape[0],48*2+w0.shape[1])
w0_p[48*2:,48*2:]=w0
w0_p=torch.tensor(w0_p)
w0_p.shape

torch.Size([224, 192])

In [87]:
b0=params["b0"]
b0.shape

torch.Size([128])

In [88]:
b0_p=np.zeros(48*2+b0.shape[0])
b0_p[48*2:]=b0
b0_p=torch.tensor(b0_p)
b0_p.shape

torch.Size([224])

In [89]:
a_p.shape,w0_p.T.shape

(torch.Size([6, 192]), torch.Size([192, 224]))

In [90]:
a_p.dtype,w0_p.dtype,b0_p.dtype

(torch.float64, torch.float64, torch.float64)

In [91]:
a0=a_p @ w0_p.T+b0_p
a0.shape

torch.Size([6, 224])

In [92]:
torch.prod(a0[:,:48]==alpha),torch.prod(a0[:,48:48*2]==beta)

(tensor(1), tensor(1))

In [93]:
params["w2"].shape

torch.Size([128, 128])

In [94]:
w2=np.eye(48*2+params["w2"].shape[0],48*2+params["w2"].shape[1])
w2[48*2:,48*2:]=params["w2"]
w2=torch.tensor(w2)
w2.shape

torch.Size([224, 224])

In [95]:
b2=torch.cat([torch.zeros(48*2),params["b2"]])
b2.shape

torch.Size([224])

In [96]:
a2=a0@w2.T+b2
a2.shape

torch.Size([6, 224])

node block

In [97]:
i,j=params["w3"].shape
w3=np.eye(48+i,48+j)
w3[48:,48:]=params["w3"]
w3=torch.tensor(w3)
w3.shape

torch.Size([176, 224])

In [98]:
b3=torch.cat([torch.zeros(48),params["b3"]])
b3.shape

torch.Size([176])

In [99]:
a3=a2@w3.T+b3
a3.shape

torch.Size([6, 176])

In [100]:
torch.prod(a3[:,:48]==alpha),torch.prod(a3[:,48:48*2]==beta)

(tensor(1), tensor(0))

In [101]:
i,j=params["w5"].shape
w5=np.eye(48+i,48+j)
w5[48:,48:]=params["w5"]
w5=torch.tensor(w5)
w5.shape

torch.Size([176, 176])

In [102]:
b5=torch.cat([torch.zeros(48),params["b5"]])
b5.shape

torch.Size([176])

In [103]:
a5=a3@w5.T+b5
a5.shape

torch.Size([6, 176])

scatter mean

In [56]:
M=torch.zeros(col.shape[0],n,dtype=torch.float64)

In [57]:
M

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)

In [58]:
for i,j in enumerate(col):
    M[i,j]=1
M=M.T
M

tensor([[0., 0., 1., 0., 1., 0.],
        [1., 0., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0.]], dtype=torch.float64)

In [60]:
m=M@a5
m.shape

torch.Size([3, 176])

In [64]:
torch.prod(m[:,:48]==x)

tensor(1)

continue within node block

In [73]:
w6=torch.tensor(params["w6"],dtype=torch.float64)
b6=torch.tensor(params["b6"],dtype=torch.float64)
a6=a5@w6.T+b6
a6.shape

/Users/indigo410/opt/anaconda3/envs/dsp/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/Users/indigo410/opt/anaconda3/envs/dsp/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


torch.Size([6, 128])

In [75]:
w8=torch.tensor(params["w8"],dtype=torch.float64)
b8=torch.tensor(params["b8"],dtype=torch.float64)
a8=a6@w8.T+b8
a8.shape

/Users/indigo410/opt/anaconda3/envs/dsp/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/Users/indigo410/opt/anaconda3/envs/dsp/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


torch.Size([6, 128])

global block

scatter mean?

In [119]:
display(a8.shape)
a8_p=a8.mean(0)

torch.Size([6, 128])

In [139]:
(torch.ones(6,dtype=torch.float64)@a8/6)

tensor([ 0.2942, -0.2485,  0.2253,  0.0817, -0.0348,  0.2023, -0.1418,  0.6224,
        -0.2384, -0.3247,  0.5302,  0.1043, -0.3219,  0.7994, -0.0656,  0.5350,
        -0.1087,  0.2923, -0.3072, -0.3681, -0.0248,  0.0594, -0.4988, -0.0653,
        -0.2231, -0.6669, -0.7339, -0.1788, -0.0791, -0.7194, -0.3306, -0.2657,
        -0.3448,  0.0409,  0.7050, -0.1336, -0.6460, -0.0289,  0.2894, -0.1160,
         0.2218, -0.1697,  0.0224, -0.3302,  0.1279, -0.0600,  0.2026,  0.2647,
         0.0845,  0.0426, -0.2317, -0.2089,  0.4994,  0.2698, -0.1798, -0.1699,
         0.8241,  0.1331, -0.0839, -0.0390,  0.7269, -0.1224,  0.3306,  0.8688,
         0.0770, -0.2528,  0.0249, -0.3981,  0.3225, -0.5872,  0.1055,  0.1216,
         0.2511,  0.2363, -0.3437, -0.0016,  0.0052,  0.2481,  0.3871,  0.3823,
         0.1270,  0.2895, -0.4254,  0.4618, -0.5040,  0.0983,  0.5255,  0.1691,
         0.1251, -0.6663,  0.2843,  0.2425,  0.9668,  0.0787, -0.2004,  0.4134,
         0.4289, -0.1550, -0.2484, -0.17

In [120]:
w9=torch.tensor(params["w9"],dtype=torch.float64)
b9=torch.tensor(params["b9"],dtype=torch.float64)
a9=a8_p@w9.T+b9
a9.shape

/Users/indigo410/opt/anaconda3/envs/dsp/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/Users/indigo410/opt/anaconda3/envs/dsp/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


torch.Size([128])

In [121]:
w11=torch.tensor(params["w11"],dtype=torch.float64)
b11=torch.tensor(params["b11"],dtype=torch.float64)
a11=a9@w11.T+b11
a11.shape

/Users/indigo410/opt/anaconda3/envs/dsp/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  """Entry point for launching an IPython kernel.
/Users/indigo410/opt/anaconda3/envs/dsp/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


torch.Size([2])

`u` and predicted class probability

In [122]:
a11

tensor([ 1.8594, -1.5062], dtype=torch.float64)

In [131]:
nn.Softmax(0)(a11)

tensor([0.9666, 0.0334], dtype=torch.float64)

model as a linear function: